In [5]:
import os

from dotenv import load_dotenv
from openai import OpenAI

In [8]:
load_dotenv(dotenv_path=".env.test")

True

In [9]:
api_key = os.environ.get("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

In [10]:
vector_store = client.beta.vector_stores.create(name="test-assistant-nb")

In [12]:
guide = client.files.create(file=open("tests/data/assistants-guide.md", "rb"), purpose="assistants")

In [13]:
guide.status

'processed'

In [15]:
guide_vector = client.beta.vector_stores.files.create(vector_store_id=vector_store.id, file_id=guide.id)

In [20]:
guide_vector

VectorStoreFile(id='file-Q5ga16Rb6QqgU8NBLaMeyJQA', created_at=1716470281, last_error=None, object='vector_store.file', status='in_progress', usage_bytes=0, vector_store_id='vs_aRrmLqwwy73B2opMGX9rRMmk')

In [21]:
import time

while guide_vector.status == "in_progress":
  print("waiting...")
  time.sleep(0.5)
  guide_vector = client.beta.vector_stores.files.retrieve(guide_vector.id, vector_store_id=guide_vector.vector_store_id)

waiting...


In [22]:
assistant = client.beta.assistants.create(
  name="OpenAI API Expert",
  description="You are a OpenAI API Expert. Refer to the knowledgebase to answer user queries.",
  model="gpt-4o-2024-05-13",
  tools=[{"type": "file_search"}],
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

In [23]:
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "What is a run step?",
    }
  ]
)

In [24]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)

In [27]:
run

Run(id='run_WlRYcqSjmKCtDBmR4JXaSCSB', assistant_id='asst_xMjce2z5beg0mLfcIBel0kU4', cancelled_at=None, completed_at=None, created_at=1716470797, expires_at=1716471397, failed_at=None, incomplete_details=None, instructions=None, last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o-2024-05-13', object='thread.run', required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_H3jEzgFNxbL6QLBLimPVlTNM', tool_choice='auto', tools=[FileSearchTool(type='file_search')], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={})

In [28]:
while run.status != "completed":
  print("waiting...")
  time.sleep(1)
  run = client.beta.threads.runs.retrieve(run.id, thread_id=thread.id)

waiting...


In [29]:
run.status

'completed'

In [31]:
messages = client.beta.threads.messages.list(thread_id=thread.id)

In [34]:
import textwrap

text = messages.data[0].content[0].text.value
wrapped_text = textwrap.fill(text, width=80)

# Print the wrapped text
print(wrapped_text)

A "run step" refers to a detailed list of actions that an Assistant takes during
a run. When an Assistant is invoked in a thread to perform tasks by calling
models and tools, each of these actions and calls are logged as run steps. The
run step object contains detailed information about the actions taken, which
allows developers to introspect and understand how the Assistant arrives at its
final results. This can include actions like creating messages or calling
tools【4:0†source】.


In [37]:
import pprint

pp = pprint.PrettyPrinter(indent=2)
pp.pprint(messages.to_dict())

{ 'data': [ { 'assistant_id': 'asst_xMjce2z5beg0mLfcIBel0kU4',
              'attachments': [],
              'content': [ { 'text': { 'annotations': [ { 'end_index': 484,
                                                          'file_citation': { 'file_id': 'file-Q5ga16Rb6QqgU8NBLaMeyJQA'},
                                                          'start_index': 472,
                                                          'text': '【4:0†source】',
                                                          'type': 'file_citation'}],
                                       'value': 'A "run step" refers to a '
                                                'detailed list of actions that '
                                                'an Assistant takes during a '
                                                'run. When an Assistant is '
                                                'invoked in a thread to '
                                                'perform tasks by calling 